In [1]:
import requests
import json
import numpy as np
import pandas as pd
from datetime import datetime 
from datetime import timedelta

In [2]:
def get_stock_rank_info(his_day, tickers, step_back_days):
    today_time=datetime.now() 
    ini_time=datetime(1970,1,1,0,0,0,0)
    start_time = today_time - timedelta(days=his_day)
    start_sec = (start_time-ini_time).days*3600*24
    end_sec = (today_time-ini_time).days*3600*24
    
    ticker_price = pd.DataFrame()
    for t in tickers:
        print("start to calculate return and rank for ticker " + t)
        site = f'https://query1.finance.yahoo.com/v7/finance/download/{t}?period1={start_sec}&period2={end_sec}&interval=1d&events=history&includeAdjustedClose=true'
        #ticker_tmp = pd.read_csv(site,parse_dates=["Date"]).set_index("Date")
        ticker_tmp = pd.read_csv(site,parse_dates=["Date"])
        ticker_tmp["Ticker"] = t
        ticker_tmp = ticker_tmp.set_index(["Date","Ticker"])
        tmp_df = ticker_tmp["Close"].to_frame()
        for i in step_back_days:
            tmp_df_step1 = tmp_df.diff(periods = i)
            a = "_" + str(i)+"_daysDiff"
            ticker_tmp = ticker_tmp.join(tmp_df_step1, on=None, how='left', lsuffix='', rsuffix=a, sort=True)
            b = "Return_" + str(i)+"_days"
            c = "Close" + a
            ticker_tmp[b] = ticker_tmp[c]/(ticker_tmp["Close"]-ticker_tmp[c]) 
        #ticker_tmp["Ticker"] = t
        ticker_price = pd.concat([ticker_price,ticker_tmp])
        
    for i in step_back_days:    
        ticker_price[str(i)+"_days_rank"] = ticker_price.groupby('Date')["Return_" + str(i)+"_days"].rank(method='first', ascending=False)
    del tmp_df, tmp_df_step1
    
    ticker_price2 = ticker_price.copy()
    
    for s in step_back_days:
        t_part_total = pd.DataFrame()
        for t in tickers:
            print("start to calculate average rank for stock " + t+" with time pace " + str(s) + " days")
            idx = pd.IndexSlice
            ticker_tmp = ticker_price2.loc[idx[:, t], :]
            ticker_tmp = ticker_tmp.sort_index(level=0,ascending=False)
            t_part = pd.DataFrame(columns= ["Date","Ticker",str(s)+"_day_rank_mean"])
            for i in range(0,len(ticker_tmp[str(s)+"_days_rank"].to_frame().dropna())-s-1):
                ticker_tmp2 = ticker_tmp.iloc[i:i+s]
                ticker_mean = ticker_tmp2[str(s)+"_days_rank"].mean()
                max_day = ticker_tmp2.index[0][0]
                t_name = t
                t_part.loc[i] = [max_day,t_name,ticker_mean]
            t_part_total = pd.concat([t_part_total,t_part])
            
        t_part_total = t_part_total.reset_index(drop=True)
        t_part_total = t_part_total.set_index(["Date","Ticker"])
        ticker_price2 = ticker_price2.join(t_part_total, on=None, how='left', lsuffix='', rsuffix='', sort=False)
    ticker_price2 = ticker_price2.sort_index(level=[1,0],ascending=False)
    return ticker_price2



In [ ]:
his_day = 1000 #define how many days we get tickers history
tickers = ["AXP","AMGN","AAPL","BA","CAT","CSCO","CVX","GS","HD","HON","IBM","INTC","JNJ","KO","JPM","MCD","MMM","MRK","MSFT","NKE","PG","TRV","UNH","CRM","VZ","V","WBA","WMT","DIS","DOW"] #define tickers interested
step_back_days = [7,30,60,90,180,365] #define time pace we want to do the return and rank
df_rank = get_stock_rank_info(his_day, tickers, step_back_days)
df_rank.to_csv("output_temp.csv")
    

start to calculate return and rank for ticker AXP
start to calculate return and rank for ticker AMGN
start to calculate return and rank for ticker AAPL
start to calculate return and rank for ticker BA
start to calculate return and rank for ticker CAT
start to calculate return and rank for ticker CSCO
start to calculate return and rank for ticker CVX
start to calculate return and rank for ticker GS
start to calculate return and rank for ticker HD
start to calculate return and rank for ticker HON
start to calculate return and rank for ticker IBM
start to calculate return and rank for ticker INTC
start to calculate return and rank for ticker JNJ
start to calculate return and rank for ticker KO
start to calculate return and rank for ticker JPM
start to calculate return and rank for ticker MCD
start to calculate return and rank for ticker MMM
start to calculate return and rank for ticker MRK
start to calculate return and rank for ticker MSFT
start to calculate return and rank for ticker NKE

In [4]:
df_rank

,,Open,High,Low,Close,Adj Close,Volume,Close_30_daysDiff,Return_30_days,Close_60_daysDiff,Return_60_days,...,30_days_rank,60_days_rank,90_days_rank,180_days_rank,365_days_rank,30_day_rank_mean,60_day_rank_mean,90_day_rank_mean,180_day_rank_mean,365_day_rank_mean
Date,Ticker,,,,,,,,,,,,,,,,,,,,,
2022-06-06,WMT,125.820000,126.300003,124.584999,124.870003,124.870003,6310608,-31.989998,-0.203940,-17.760002,-0.124518,...,29.0,27.0,17.0,21.0,25.0,16.200000,10.000000,14.500000,16.405556,NaN
2022-06-03,WMT,126.669998,127.419998,125.089996,125.320000,125.320000,6096500,-34.549995,-0.216113,-14.140007,-0.101391,...,29.0,25.0,19.0,20.0,24.0,15.466667,9.766667,14.555556,16.427778,NaN
2022-06-02,WMT,125.570000,127.529999,124.330002,127.510002,127.510002,7738100,-32.120003,-0.201215,-11.230003,-0.080943,...,29.0,26.0,21.0,20.0,25.0,14.600000,9.566667,14.577778,16.461111,NaN
2022-06-01,WMT,128.500000,128.919998,124.680000,125.449997,125.449997,9511600,-32.199997,-0.204250,-16.220001,-0.114491,...,29.0,27.0,21.0,21.0,24.0,13.766667,9.316667,14.555556,16.494444,NaN
2022-05-31,WMT,127.459999,129.899994,127.419998,128.630005,128.630005,12304100,-27.250000,-0.174814,-14.190002,-0.099356,...,29.0,26.0,18.0,20.0,23.0,13.000000,9.033333,14.533333,16.522222,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,AAPL,72.779999,73.492500,72.029999,72.449997,71.194130,146266000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,AAPL,71.205002,72.495003,71.175003,72.477501,71.221146,93121200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-24,AAPL,71.172501,71.222504,70.730003,71.067497,69.835602,48478800,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
